### Week 6: Evolutionary Computation
```
- Advanced Machine Learning, Innopolis University 
- Professor: Muhammad Fahim 
- Teaching Assistant: Gcinizwe Dlamini
```
<hr>


```
Lab Plan
    1. Evolution Theory
    2. Genetic Algorithms
    3. Manual Implementation
    4. DEAP
    5. Applications of HMM, RNN and CNN (Preparation for Midterm)
```

<hr>

## 2. Genetic Algorithms

Similarly to the tomato we will make a new optimization technique for non smooth non convex problems.

![alt text](https://www.researchgate.net/profile/Joachim_Wegener/publication/2485300/figure/fig2/AS:669985449975833@1536748387334/The-Process-of-Evolutionary-Computation.png)

In order to make our own Genetic algorithm we need to define the following:

1. Representation scheme (e.g genotype, phenotype, …etc)
2. Mating operators (e.g crossover)
3. Mutating operators (e.g bit flipping)
4. Fitness metric
5. Selection strategy (e.g tournament selection)
6. Evolution strategy (e.g mu,lambda)



## 3. Manual Implementation

Word Matching:

guess the word 'ch muhammad awais'

fitness function is = corrcet placement.

Encoding schema:
string with a specific length.

In [1]:
import random
import string
from random import shuffle

import numpy as np

genes = string.ascii_lowercase+' '

In [2]:
correct_guess = 'ch muhammad awais'
gene_length = len(correct_guess)

def fitness(guess):
    result = 0
    for i in range(len(correct_guess)):
        if correct_guess[i] == guess[i]:
            result +=1 * 28
    for c in guess:
        if c in correct_guess:
            result+=1
    return result

    # return random.randint(0,100)

def get_fitness(population):
    fitnesses = []
    for ind in population:
        fitnesses.append(fitness(ind))
    return fitnesses

def sample():
    return ''.join(random.choice(genes) for i in range(gene_length))

print ("Random String is ", sample(),gene_length )

Random String is  mivgvod tfyhkibsy 17


In [3]:
population_size = 100

def populate():
    population = []
    for i in range(population_size):
        population.append(sample())

    return population


In [4]:
from random import randint

def cross_over(individual1,individual2):
    cross_point = randint(0, gene_length)
    indiv1_part1, indiv1_part2 = individual1[:cross_point],individual1[cross_point:]
    indiv2_part1, indiv2_part2 = individual2[:cross_point],individual2[cross_point:]

    child1 = indiv1_part1+indiv2_part2
    child2 = indiv2_part1+indiv1_part2
    return child1,child2

def mutation(individual):
    mutation_point = randint(1, gene_length-1)
    individual = list(individual) 
    individual[mutation_point]=random.choice(genes)
    return "".join(individual)

def avg_fitness(fitnesses):
    return np.mean(fitnesses)

![alt text](https://www.tutorialspoint.com/genetic_algorithms/images/roulette_wheel_selection.jpg)

In [5]:
def select(samples):
    fitnesses = get_fitness(samples)
    best = np.array(fitnesses).argsort()[-len(samples)//2:][::-1]
    best = np.array(samples)[best]
    shuffle(best)
    return best

In [6]:
def take_best(parents,children,top = population_size):
    parents = list(parents)
    children = list(children)
    options = parents+children
    fitnesses = get_fitness(options)
    best = np.array(fitnesses).argsort()[-top:][::-1]
    best = np.array(options)[best]
    shuffle(best)
    return best

In [7]:
stop = 500
top = 10
log = 10
p_mutation = 0.01
population = populate()

for i in range(stop):
    
    parents1 = select(population)
    parents2 = select(population)
    # print(parents1)
    children = []
    for p1,p2 in zip(parents1,parents2):
        c1, c2 = cross_over(p1,p2)
        if random.uniform(0, 1)<p_mutation:
            c1 = mutation(c1)
        if random.uniform(0, 1)<p_mutation:
            c2 = mutation(c2)
        children.append(c1)
        children.append(c2)
    population = take_best(population,children)
    avg = avg_fitness(get_fitness(population))
    if i % log:
        print('average on iteration',i,'is',avg,take_best(population,population,1)[0])

average on iteration 1 is 74.26 khhrphgsmarqgrxys
average on iteration 2 is 95.46 khhrphgsmarqgrxys
average on iteration 3 is 118.0 jmbauresmaiqawulu
average on iteration 4 is 132.76 khhrphgsmac aepps
average on iteration 5 is 156.29 jmbauresmaiqawuis
average on iteration 6 is 175.45 khbauhgsmarqawxys
average on iteration 7 is 201.26 khhrphgsmac a cis
average on iteration 8 is 218.62 ihhauhgsmaiqawuis
average on iteration 9 is 240.24 khbauhgmmaiqawuis
average on iteration 11 is 272.19 khbauhgmmac awuis
average on iteration 12 is 287.62 ihhauhgmmac awuis
average on iteration 13 is 305.66 ihhauhgmmai awuis
average on iteration 14 is 322.69 ihhauhgmmai awuis
average on iteration 15 is 323.81 ihhauhgmmac awuis
average on iteration 16 is 324.0 ihhauhgmmac awuis
average on iteration 17 is 324.0 ihhauhgmmac awuis
average on iteration 18 is 324.0 ihhauhgmmac awuis
average on iteration 19 is 324.0 ihhauhgmmac awuis
average on iteration 21 is 324.0 ihhauhgmmai awuis
average on iteration 22 is 32

In [8]:
take_best(population,population,top)

array(['ih muhammad awais', 'ih muhammad awais', 'ih muhammad awais',
       'ih muhammad awais', 'ih muhammad awais', 'ih muhammad awais',
       'ih muhammad awais', 'ih muhammad awais', 'ih muhammad awais',
       'ih muhammad awais'], dtype='<U17')

## 4. DEAP

![alt text](https://deap.readthedocs.io/en/master/_images/deap_long.png)

DEAP is a novel evolutionary computation framework for rapid prototyping and testing of ideas. It seeks to make algorithms explicit and data structures transparent. It works in perfect harmony with parallelisation mechanism such as multiprocessing and SCOOP. The following documentation presents the key concepts and many features to build your own evolutions.

In [9]:
!pip3 install deap

     |████████████████████████████████| 163kB 10.8MB/s 


The problem now is called **One Max Problem**:

The problem itself is both very simple and widely used in the evolutionary computational community. We will create a population of individuals consisting of integer vectors randomly filled with 0 and 1. Then we let our population evolve until one of its members contains only 1 and no 0 anymore.

In [10]:
import random

from deap import base
from deap import creator
from deap import tools

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Attribute generator 
# define 'attr_bool' to be an attribute ('gene')
# which corresponds to integers sampled uniformly
# from the range [0,1] (i.e. 0 or 1 with equal
# probability)

toolbox.register("attr_bool", random.randint, 0, 1)

# Structure initializers
# define 'individual' to be an individual
# consisting of 100 'attr_bool' elements ('genes')

toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 100)

# define the population to be a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# the goal ('fitness') function to be maximized
def evalOneMax(individual):
    return sum(individual),


# Operator registration
# register the goal / fitness function
toolbox.register("evaluate", evalOneMax)

# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
toolbox.register("select", tools.selTournament, tournsize=3)

def main():
    random.seed(64)

    # create an initial population of 300 individuals (where each individual is a list of integers)
    pop = toolbox.population(n=300)

    # CXPB  is the probability with which two individuals are crossed
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.2
    
    print("Start of evolution")
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    
    print(f"  Evaluated {len(pop)} individuals" )

    # Extracting all the fitnesses 
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0
    
    # Begin the evolution
    while max(fits) < 100 and g < 1000:
        # A new generation
        g = g + 1
        print(f"-- Generation {g} --")
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
    
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        print(f"  Evaluated {len(invalid_ind)} individuals")
        
        # The population is entirely replaced by the offspring
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print(f"  Min {min(fits)}")
        print(f"  Max {max(fits)}")
        print(f"  Avg {mean}")
        print(f"  Std {std}")
    
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print(f"Best individual is {best_ind}", best_ind.fitness.values)

if __name__ == "__main__":
    main()

Start of evolution
  Evaluated 300 individuals
-- Generation 1 --
  Evaluated 181 individuals
  Min 44.0
  Max 66.0
  Avg 54.833333333333336
  Std 4.349584909952722
-- Generation 2 --
  Evaluated 191 individuals
  Min 47.0
  Max 68.0
  Avg 58.45666666666666
  Std 3.455641120769904
-- Generation 3 --
  Evaluated 199 individuals
  Min 52.0
  Max 68.0
  Avg 60.95333333333333
  Std 2.9024970092816367
-- Generation 4 --
  Evaluated 167 individuals
  Min 47.0
  Max 71.0
  Avg 62.96
  Std 2.907186497858939
-- Generation 5 --
  Evaluated 175 individuals
  Min 57.0
  Max 73.0
  Avg 64.99
  Std 2.8489588741621903
-- Generation 6 --
  Evaluated 168 individuals
  Min 58.0
  Max 74.0
  Avg 66.93333333333334
  Std 2.8051539866624524
-- Generation 7 --
  Evaluated 187 individuals
  Min 59.0
  Max 76.0
  Avg 68.91666666666667
  Std 2.826609669236565
-- Generation 8 --
  Evaluated 171 individuals
  Min 62.0
  Max 76.0
  Avg 70.88666666666667
  Std 2.4455038108513407
-- Generation 9 --
  Evaluated 155 i